In [9]:
import plotly.figure_factory as ff
import pandas as pd

In [29]:
#calls 
calls = ['https://data.police.uk/api/crimes-street/all-crime?poly=53.933024,%20-1.949214:53.936022,%20-2.049110:54.016624,%20-1.954005:54.009636,%20-2.076643&date=2019-09',
         'https://data.police.uk/api/crimes-street/all-crime?poly=53.933024,%20-1.949214:53.936022,%20-2.049110:54.016624,%20-1.954005:54.009636,%20-2.076643&date=2019-08',
         'https://data.police.uk/api/crimes-street/all-crime?poly=53.933024,%20-1.949214:53.936022,%20-2.049110:54.016624,%20-1.954005:54.009636,%20-2.076643&date=2019-07',
         'https://data.police.uk/api/crimes-street/all-crime?poly=53.933024,%20-1.949214:53.936022,%20-2.049110:54.016624,%20-1.954005:54.009636,%20-2.076643&date=2019-06',
         'https://data.police.uk/api/crimes-street/all-crime?poly=53.933024,%20-1.949214:53.936022,%20-2.049110:54.016624,%20-1.954005:54.009636,%20-2.076643&date=2019-05',
         'https://data.police.uk/api/crimes-street/all-crime?poly=53.933024,%20-1.949214:53.936022,%20-2.049110:54.016624,%20-1.954005:54.009636,%20-2.076643&date=2019-04']

# Create an initial empty data frame
df = pd.DataFrame()
# Make the consecutive calls
for i, call in enumerate(calls):
    # Create the new DataFrame from the data you got
    df_new = pd.read_json(call, orient='records')
    # Merge it with the current data frame
    df = pd.concat([df, df_new])
    
df.info()
df.sample(5)


<class 'pandas.core.frame.DataFrame'>
Int64Index: 550 entries, 0 to 89
Data columns (total 9 columns):
category            550 non-null object
context             550 non-null object
id                  550 non-null int64
location            550 non-null object
location_subtype    550 non-null object
location_type       550 non-null object
month               550 non-null object
outcome_status      392 non-null object
persistent_id       550 non-null object
dtypes: int64(1), object(8)
memory usage: 43.0+ KB


,category,context,id,location,location_subtype,location_type,month,outcome_status,persistent_id
76,violent-crime,,77289597,"{'latitude': '53.939454', 'street': {'id': 105...",,Force,2019-09,"{'category': 'Under investigation', 'date': '2...",2029dc6e8b578521dbdc3dfe7618a1df7937288acd1b3c...
20,anti-social-behaviour,,76772554,"{'latitude': '53.957438', 'street': {'id': 105...",,Force,2019-08,None,f8f3abc9d7f03f5d057e166a12356bc783e5ad9cee68ee...
7,anti-social-behaviour,,75426221,"{'latitude': '53.956440', 'street': {'id': 105...",,Force,2019-06,None,5279ac420de6c649888485d4b5a7c7f5e0532c02917fe7...
32,anti-social-behaviour,,76765723,"{'latitude': '53.959217', 'street': {'id': 105...",,Force,2019-08,None,515c87e85d0170e8b74f0ed02deff4d016e7675187e4ed...
31,anti-social-behaviour,,76777117,"{'latitude': '53.953203', 'street': {'id': 105...",,Force,2019-08,None,cd90ee59ee76b199deb720b07bab6afe353050d8cbc3d8...


In [45]:
#attempt to get the nested dictionary to display
from pandas.io.json import json_normalize

flat = 'https://data.police.uk/api/crimes-street/all-crime?poly=53.933024,%20-1.949214:53.936022,%20-2.049110:54.016624,%20-1.954005:54.009636,%20-2.076643&date=2019-04'

json_normalize(flat, 'location', ['latitude','longitude', 'name'])

TypeError: string indices must be integers

In [53]:
#another atempt to view the nested dictionaries
dftest = df[['location']]
dftest

KeyError: "None of [Index(['location'], dtype='object')] are in the [columns]"

To do list:

1) Put the data into github and read the CSV from there so that sam can access it
2) make sure the data is fully readable, removing the spaces commas in the thousands. The thousand commas can be removed during the read CSV and the spaces can be removed with a replace function.
3) use plotly to make the map

call the data from an API
Probably can call the location id's from an api as well